# Programming Project 3: The Hartree-Fock Self-Consistent Field Method 

The purpose of this project is to provide a deeper understanding of Hartree-Fock theory by demonstrating a simple implementation of the self-consistent-field method for restricted Hartree-Fock.

A main ingredient of all quantum chemical methods are the molecular integrals over all basis functions. Instead of providing them, we expect you to use a third-party library to generate the integrals for you. You will only be given the geometries of the test systems, which are given inside the `input` folder. The coordinates in these files are given in bohr, and the basis set you're expected to use is STO-3G.

You should generate the required integrals using [Psi4](https://github.com/psi4/psi4numpy/tree/master/Tutorials/01_Psi4NumPy-Basics): [Psi4Numpy](https://github.com/psi4/psi4numpy) is installed in the gqcg virtual environment.


You can check your implementation with the numerical results provided in the `solutions` directory. Note that not all results will be exactly the same, as for example the number of iterations and intermediate results may vary. The end result, however, should be the same.

## Step 1: Identifying the Molecule

In the first step, the Psi4 options are set and you should define your molecule object.

## Step 2: Prerequisite calculations

In this step, you'll use Psi4 to generate the nuclear repulsion energy and the one- and two-electron integrals.

### Step 2.1: The nuclear repulsion energy

Use Psi4 to calculate the nuclear repulsion energy of the molecule.

### Step 2.2: The one-electron integrals

Use Psi4 to calculate the overlap, kinetic energy, nuclear-attraction energy and core Hamiltonian matrices. The respective equations are given below.

$$S_{\mu \nu} \equiv \int \phi_\mu^*({\mathbf r}) \phi_{\nu}({\mathbf r}) d{\mathbf r}$$

$$T_{\mu \nu} \equiv \int \phi_\mu^*({\mathbf r}) \left( -\frac{1}{2} \nabla^2_{\mathbf r} \right) \phi_\nu({\mathbf r}) d{\mathbf r}$$

$$V_{\mu \nu} \equiv \int \phi_\mu^*({\mathbf r}) \left( -\sum_A^N \frac{Z}{r_A} \right) \phi_\nu({\mathbf r}) d{\mathbf r}$$

$$H^{\rm core}_{\mu \nu} = T_{\mu \nu} + V_{\mu \nu}$$
    

### Step 2.3: The two-electron integrals

Use Psi4 to calculate the two-electron repulsion integrals. The two-electron integrals (using chemist's notation) are given by:

$$(\mu \nu | \lambda \sigma) \equiv \int \phi_\mu^*({\mathbf r}_1) \phi_\nu({\mathbf r}_1) \, r_{12}^{-1} \, \phi_\lambda^*({\mathbf r}_2) \phi_\sigma({\mathbf r}_2) d{\mathbf r}_1 d{\mathbf r}_2 \, .$$

_Note that Psi4 returns a dense 4-dimensional array for these integrals. This means that the eight-fold permutational symmetry of the real integrals remains unused and thus we are using 8 times too much memory to store the two-electron integrals. This could be a bottleneck in the size of the molecules that can be treated using RHF theory. You might be smart and use a one-dimensional representation of the unique integrals in combination with an index function that maps the quadruple $(\mu, \nu, \lambda, \sigma)$ to the corresponding element of the one-dimensional array, but that would be overkill for this exercise._

## Step 3: The initial (guess) density matrix

Since the SCF procedure is an iterative algorithm, we should supply an initial guess for the coefficient matrix or density matrix. Often, we diagonalize the core Hamiltonian to obtain this initial guess. The initial coefficient matrix $\mathbf{C}_0$ is then given by solving the generalized eigenvalue problem

$$\mathbf{H}_{\text{core}} \mathbf{C}_0 = \mathbf{S} \mathbf{C}_0 \mathbf{\epsilon}_0 \, .$$

The initial density matrix (in AO basis) can then be calculated using the general formula that links the RHF coefficient matrix to its corresponding density matrix:

$$\mathbf{D}^{\text{AO}} = \mathbf{C} \mathbf{D}^{\text{MO}} \mathbf{C}^{\text{T}} \, ,$$

with $\mathbf{D}^{\text{MO}}$ the RHF density matrix expressed in MO basis, which is a diagonal matrix containing $2$s on the diagonal corresponding to occupied indices and $0$s elsewhere.

## Step 4: Updating the Fock matrix

Start the SCF iterative procedure by building a new Fock matrix (in AO basis), using the current AO density matrix $\mathbf{D}^{\text{AO}}$:

$$F_{\mu \nu} = H^{\text{core}}_{\mu \nu} + \sum_{\lambda \sigma} D^{\text{AO}}_{\lambda \sigma} \left[ (\mu\nu | \lambda\sigma) - \frac{1}{2} (\mu\lambda|\nu\sigma) \right]\, ,$$

where the double-summation runs over all the AOs.

## Step 5:  The SCF energy


The SCF electronic energy is computed using the current AO density and Fock and core Hamiltonian matrices as:

$$E_{\text{elec}} = \frac{1}{2} \sum_{\mu\nu} D_{\mu \nu} \left( H^{\text{core}}_{\mu\nu} + F_{\mu\nu} \right) \, ,$$

so that the total energy can be calculated as the sum of the electronic energy and the nuclear repulsion energy:

$$E_{\text{total}} = E_{\text{elec}} + E_{\text{nuc}} \, .$$

## Step 6: Test for Convergence

In order to determine if the SCF procedure has converged, test both the energy and the density for convergence:

$$\Delta E = E^i_{\text{elec}} - E^{i-1}_{\text{elec}} < \delta_1$$

$$\text{RMS}(\mathbf{D}) = \left[ \sum_{\mu\nu} \left( D^i_{\mu\nu} - D^{i-1}_{\mu\nu} \right)^2 \right]^{1/2} < \delta_2 \, ,$$

where $\delta_1$ and $\delta_2$ are thresholds for convergence. Often used thresholds are $\delta_2 = 10^{-4}$ for the densities and $\delta_2 = 10^{-6}$ for the energies. Note that the density criterion is much more severe (why?).

If the difference in consecutive SCF energy and the root-mean-squared difference in consecutive densities do not fall below the prescribed thresholds, restart the SCF procedure. This means that, instead of diagonalizing the core Hamiltonian, you should solve the generalized eigenvalue problem for the current Fock matrix $\mathbf{F}$:

$$\mathbf{F} \mathbf{C} = \mathbf{S} \mathbf{C} \mathbf{\epsilon} \, ,$$

yielding a new coefficient matrix $\mathbf{C}$. You can then form a new density matrix, as described in step 3 and continue the iterative procedure until convergence is obtained!

## Bonus

If you'd like to introduce yourself with GQCG's own quantum chemical library, [GQCP](https://github.com/GQCG/gqcp), you may also use its Python bindings to generate the integrals. Which ever library you use, the integrals should be the same!